## Deliverable 2. Create a Customer Travel Destinations Map.

In [108]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [109]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Hermanus,-34.4187,19.2345,64.00,88,0,4.00,clear sky,ZA
1,1,Biltine,14.5333,20.9167,82.27,7,5,11.54,clear sky,TD
2,2,Roma,43.2128,-75.4557,69.80,26,40,3.96,scattered clouds,US
3,3,Vaini,-21.2000,-175.2000,78.80,78,20,5.75,few clouds,TO
4,4,San Patricio,28.0170,-97.5169,91.40,58,40,9.22,scattered clouds,US
5,5,Uruzgan,32.8333,66.0000,54.88,67,78,3.02,broken clouds,AF
6,6,Zalari,53.5584,102.5065,31.96,73,100,7.70,overcast clouds,RU
7,7,Oranjemund,-28.5500,16.4333,70.90,48,0,3.65,clear sky,NaN
8,8,Ostrovnoy,68.0531,39.5131,29.44,87,64,26.40,broken clouds,RU
9,9,Punta Arenas,-53.1500,-70.9167,42.80,81,75,21.85,light rain,CL


In [110]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
lowest_temp = input("Enter lowest temperature: ")
minimum_temp = float(lowest_temp)

highest_temp = input("Enter maximum temperature: ")
maximum_temp = float(highest_temp)


Enter lowest temperature: 50
Enter maximum temperature: 60


In [111]:
type(minimum_temp)
type(maximum_temp)

float

In [119]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_range = city_data_df.loc[((city_data_df['Max Temp']) >= minimum_temp) & ((city_data_df['Max Temp']) < maximum_temp)]

temp_range.dtypes

#print(len(temp_range))

City_ID                  int64
City                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Country                 object
dtype: object

In [120]:
# 4a. Determine if there are any empty rows.
temp_range.isnull().sum()
#temp_range['Lat'].value_counts(dropna=False).count()


City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                0
dtype: int64

In [121]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_temp_range = temp_range.dropna()

city_data_range_df = pd.DataFrame(city_data_temp_range)
city_data_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
5,5,Uruzgan,32.8333,66.0000,54.88,67,78,3.02,broken clouds,AF
14,14,Lakes Entrance,-37.8811,147.9810,52.00,75,8,11.61,clear sky,AU
15,15,New Norfolk,-42.7826,147.0587,53.01,85,100,3.00,overcast clouds,AU
25,25,Jamestown,42.0970,-79.2353,57.00,44,1,6.91,clear sky,US
31,31,Mataura,-46.1927,168.8643,51.01,97,98,1.01,overcast clouds,NZ
...,...,...,...,...,...,...,...,...,...,...
547,547,Chester,40.1254,-75.8749,59.00,88,90,4.61,overcast clouds,US
548,548,Malartic,48.1335,-78.1328,59.00,31,40,2.30,scattered clouds,CA
552,552,Bar Harbor,44.3876,-68.2039,55.40,47,1,8.05,clear sky,US
554,554,Yiyang,28.5892,112.3283,54.37,95,100,4.79,overcast clouds,CN


In [124]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_range_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Uruzgan,AF,54.88,broken clouds,32.8333,66.0000,
14,Lakes Entrance,AU,52.00,clear sky,-37.8811,147.9810,
15,New Norfolk,AU,53.01,overcast clouds,-42.7826,147.0587,
25,Jamestown,US,57.00,clear sky,42.0970,-79.2353,
31,Mataura,NZ,51.01,overcast clouds,-46.1927,168.8643,
43,Spornoye,RU,51.80,clear sky,50.8448,37.2913,
64,Hobart,AU,54.00,broken clouds,-42.8794,147.3294,
71,Soma,TR,57.00,clear sky,39.1855,27.6094,
77,Aleksandrov Gay,RU,54.30,clear sky,50.1470,48.5704,
101,Bredasdorp,ZA,57.20,clear sky,-34.5322,20.0403,


In [125]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.   
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [145]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)

clean_hotel_df = hotel_df.dropna()


In [146]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [148]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
14,-37.8811,147.9810
15,-42.7826,147.0587
25,42.0970,-79.2353
31,-46.1927,168.8643
64,-42.8794,147.3294
...,...,...
547,40.1254,-75.8749
548,48.1335,-78.1328
552,44.3876,-68.2039
554,28.5892,112.3283


In [149]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))